In [1]:
import torch
import utils
import os
import numpy as np
from collections import OrderedDict
from vgg_face_dag import vgg_face_dag

SKIP_LAYERS = utils.SKIP_LAYERS

device = torch.device('cuda')
vgg_weight = './vgg_face_dag.pth'

# ami model
attribute_model = vgg_face_dag(vgg_weight)
attribute_model.to(device)
attribute_model.eval()
attribute_model.set_ami_params(100., 2.15, 60.)
attribute_model.register_my_hook(skip_layers=SKIP_LAYERS, ami_data=utils.load_neuron_set_lists())

# original model
vgg_net = vgg_face_dag(vgg_weight)
vgg_net.to(device)
vgg_net.eval()

register hook for conv1_1
register hook for conv1_2
register hook for pool1
register hook for conv2_1
register hook for conv2_2
register hook for pool2
register hook for conv3_1
register hook for conv3_2
register hook for conv3_3
register hook for pool3
register hook for conv4_1
register hook for conv4_2
register hook for conv4_3
register hook for pool4
register hook for conv5_1
register hook for conv5_2
register hook for conv5_3
register hook for pool5
register hook for fc6
register hook for fc7


Vgg_face_dag(
  (conv1_1): Conv2d(3, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu1_1): ReLU(inplace=True)
  (conv1_2): Conv2d(64, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu1_2): ReLU(inplace=True)
  (pool1): MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
  (conv2_1): Conv2d(64, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu2_1): ReLU(inplace=True)
  (conv2_2): Conv2d(128, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu2_2): ReLU(inplace=True)
  (pool2): MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
  (conv3_1): Conv2d(128, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu3_1): ReLU(inplace=True)
  (conv3_2): Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu3_2): ReLU(inplace=True)
  (conv3_3): Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu3_3): ReLU(inplace=True)
  (pool3)

In [2]:
vgg_root   = '../../data/vgg_face_caffe/'
vgg_names  = utils.read_list(vgg_root + 'names.txt')

In [8]:
attack_path = '../../data/attacks/patch_next/'

img_count = 0
adv_count = 0

with torch.no_grad():
    for img_name in os.listdir(attack_path):
        img_name  = img_name.strip()
        img_path  = attack_path + img_name

        prob_original  = vgg_net(utils.get_data(img_path).to(device))
        prob_attribute = attribute_model(utils.get_data(img_path).to(device))
        id_original    = torch.max(prob_original, 1)[1].item()
        id_attribute   = torch.max(prob_attribute, 1)[1].item()

        id_gold = utils.get_identity(img_name, vgg_names)
        
        if id_gold != id_original:
            img_count += 1
        
            ### consistency observer for detecting adversarial samples ###
            if id_original != id_attribute:
                adv_count += 1

            print('{:3} Gold-Original-Attribute: {:4}-{:4}-{:4}  |  Adversary: {:.4f} [{:3}/{:3}]'
                  .format(img_count, id_gold, id_original, id_attribute, 
                          1.0*adv_count/img_count, adv_count, img_count))

  1 Gold-Original-Attribute: 2489- 619- 619  |  Adversary: 0.0000 [  0/  1]
  2 Gold-Original-Attribute:  737- 738-2216  |  Adversary: 0.5000 [  1/  2]
  3 Gold-Original-Attribute: 1859-1860- 511  |  Adversary: 0.6667 [  2/  3]
  4 Gold-Original-Attribute: 1944-1743-1945  |  Adversary: 0.7500 [  3/  4]
  5 Gold-Original-Attribute: 1124-1189-1657  |  Adversary: 0.8000 [  4/  5]
  6 Gold-Original-Attribute: 2189- 159- 525  |  Adversary: 0.8333 [  5/  6]
  7 Gold-Original-Attribute:   70-1335- 109  |  Adversary: 0.8571 [  6/  7]
  8 Gold-Original-Attribute:  586-1859-1859  |  Adversary: 0.7500 [  6/  8]
  9 Gold-Original-Attribute:  220- 221- 836  |  Adversary: 0.7778 [  7/  9]
 10 Gold-Original-Attribute: 1992-  78-  78  |  Adversary: 0.7000 [  7/ 10]
 11 Gold-Original-Attribute: 1664-1072-1145  |  Adversary: 0.7273 [  8/ 11]
 12 Gold-Original-Attribute:  134- 658- 863  |  Adversary: 0.7500 [  9/ 12]
 13 Gold-Original-Attribute:  237- 251- 251  |  Adversary: 0.6923 [  9/ 13]
 14 Gold-Ori